# Model testbench


In this jupyter notebook I have all the model tests documented. Written in Python3 and SKlearn. Don't "run all" this notebook. It's computationally complex and can take a very long time to complete.

Dependences: \
-Numpy  \
-Pandas \
-Matplotlib \
-ScyPy \
-Sklearn \
-Lightgbm 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier,NeighborhoodComponentsAnalysis, NearestCentroid
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA, KernelPCA, IncrementalPCA
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
import lightgbm as lgb
from scipy.fft import fft, ifft
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB, CategoricalNB

In [57]:
dataset = pd.read_csv("dataset_clone.csv")
y=dataset.iloc[:,-1]
X=dataset.iloc[:,:-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

## Basic classifications (baseline)

In [59]:
clf = LogisticRegression(random_state=0, penalty='none').fit(X_train, y_train)
clf.score(X_test, y_test)

0.2932330827067669

In [60]:
clf2=LinearSVC(penalty='l2',random_state=0, tol=1e-5,max_iter=9000,C=1,loss='squared_hinge').fit(X_train,y_train)
clf2.score(X_test, y_test), clf2.score(X_train,y_train)

(0.3007518796992481, 1.0)

In [62]:
clf3 = KNeighborsClassifier(n_neighbors=7).fit(X_train, y_train)
clf3.score(X_test, y_test)

0.23308270676691728

In [ ]:
sc=0
iteration=0
for neig in range(1,200):
    clf3 = KNeighborsClassifier(n_neighbors=neig).fit(X_train_PCA, y_train)
    if sc<clf3.score(X_test_PCA,y_test):
        sc=clf3.score(X_test_PCA,y_test)
        iteration=neig
print (sc, iteration)

In [ ]:
red5=PCA(.90)
X_train_PCA=red5.fit_transform(X_train)
X_test_PCA=red5.transform(X_test)
sc=0
iteration=0
for neig in range(1,200):
    clf3 = KNeighborsClassifier(n_neighbors=neig).fit(X_train_PCA, y_train)
    if sc<clf3.score(X_test_PCA,y_test):
        sc=clf3.score(X_test_PCA,y_test)
        iteration=neig
print (sc, iteration)

In [ ]:
clf4 = LogisticRegression(random_state=0, max_iter=10000)
params=([{'penalty': ['none'], 'solver': ['newton-cg', 'lbfgs', 'sag','saga']},
        {'penalty': ['elasticnet'], 'solver': ['saga']},
        {'penalty': ['l2'], 'solver': ['newton-cg', 'lbfgs', 'sag','saga','liblinear']},
        {'penalty': ['l1'], 'solver': ['liblinear', 'saga']},])
gs=GridSearchCV(clf8,params).fit(X_train,y_train)

## SVC

In [ ]:
clf4=LinearSVC(penalty='l2',random_state=0, tol=1e-5,max_iter=9000,C=1,loss='squared_hinge').fit(X_train,y_train)
clf7.score(X_test, y_test)

In [99]:
clf6 = SVC(random_state=1, kernel='rbf', C=80, gamma=0.0001).fit(X_train,y_train)
clf6.score(X_test, y_test)
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]
gs=GridSearchCV(clf6,param_grid).fit(X_train,y_train)
gs.best_estimator_, gs.best_params_, gs.best_score_

(SVC(C=100, gamma=0.0001, random_state=1),
 {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'},
 0.45505023796932836)

In [ ]:
param_grid = [
  {'C': [40,100,400], 'gamma': [0.001, 0.0001, 0.00001], 'kernel': ['rbf']},
 ]

gs2=GridSearchCV(clf6,param_grid).fit(X,y)
gs2.best_estimator_, gs2.best_params_, gs2.best_score_

In [ ]:
param_grid3= [
  {'C': [1,10,80,100,400,1000], 'loss': ['hinge','squared_hinge']},]
gs3=GridSearchCV(LinearSVC(), param_grid3).fit(X,y)

In [64]:
red=PCA(.90)
X_train_PCA=red.fit_transform(X_train)
X_test_PCA=red.transform(X_test)

clf5 = SVC(random_state=1, kernel='rbf', C=80, gamma=0.0001).fit(X_train,y_train)
clf5.score(X_test, y_test)

0.39097744360902253

In [58]:
red2=KernelPCA(.90)
X_train_KPCA=red.fit_transform(X_train)
X_test_KPCA=red.transform(X_test)
param_grid = [{'C': [40,100,400], 'gamma': [0.001, 0.0001, 0.00001], 'kernel': ['rbf']},]
gs4=GridSearchCV(clf5,param_grid).fit(X_train_KPCA,y_train)
gs4.best_estimator_, gs4.best_score_

(SVC(C=100, gamma=0.0001, random_state=1), 0.43796932839767316)

In [68]:
red2=IncrementalPCA(.90)
X_train_IPCA=red2.fit_transform(X_train)
X_test_IPCA=red2.transform(X_test)
clf6=SVC()
param_grid = [{'C': [40,100,400], 'gamma': [0.001, 0.0001, 0.00001], 'kernel': ['rbf']},
              {'C': [40, 100, 400], 'kernel': ['linear'], 'gamma': [0.001, 0.0001, 0.00001]},
              {'C': [40, 100, 400], 'kernel': ['poly'], 'gamma': [0.001, 0.0001, 0.00001],},]

gs5=GridSearchCV(clf6,param_grid).fit(X_train_IPCA,y_train)
gs5.best_estimator_, gs5.best_score_, gs5.best_params_

(SVC(C=100, gamma=0.0001),
 0.408197139938713,
 {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'})

In [165]:
red3=KernelPCA(.90)
X_train_KPCA=red3.fit_transform(X_train)
X_test_KPCA=red3.transform(X_test)
clf7=SVC()
param_grid = [{'C': [40,100,400], 'gamma': [0.001, 0.0001, 0.00001], 'kernel': ['rbf']},
              {'C': [40, 100, 400], 'kernel': ['linear'], 'gamma': [0.001, 0.0001, 0.00001]},
              {'C': [40, 100, 400], 'kernel': ['poly'], 'gamma': [0.001, 0.0001, 0.00001],},]

gs=GridSearchCV(clf7,param_grid).fit(X_train_PCA,y_train)
gs.best_estimator_, gs.best_score_, gs.best_params_

(SVC(C=100, gamma=0.0001),
 0.408197139938713,
 {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'})

In [69]:
red4=PCA(.90)
sc=StandardScaler()
X_train_sc=sc.fit_transform(X_train)
X_test_sc=sc.transform(X_test)
X_train_PCA=red4.fit_transform(X_train_sc)
X_test_PCA=red4.transform(X_test_sc)
sc=StandardScaler()
sc.fit_transform(X_train)
sc.transform(X_test)
clf6=SVC()
param_grid = [{'C': [0.1, 40, 100, 400], 'gamma': [0.001, 0.0001, 0.00001], 'kernel': ['rbf']},
              {'C': [0.1, 40, 100, 400], 'kernel': ['linear'], 'gamma': [0.001, 0.0001, 0.00001]},
              {'C': [0.1, 40, 100, 400], 'kernel': ['poly'], 'gamma': [0.001, 0.0001, 0.00001],},]

gs=GridSearchCV(clf6,param_grid).fit(X_train_PCA,y_train)
gs.best_estimator_, gs.best_score_, gs.best_params_

(SVC(C=100, gamma=1e-05),
 0.42527763088313064,
 {'C': 100, 'gamma': 1e-05, 'kernel': 'rbf'})

In [65]:
red4=PCA()
sc2=StandardScaler()
X_train_sc=sc2.fit_transform(X_train)
X_test_sc=sc2.transform(X_test)
X_train_PCA=red4.fit_transform(X_train_sc)
X_test_PCA=red4.transform(X_test_sc)

clf6=LinearSVC(max_iter=5000)
param_grid = [{'C': [0.1, 40, 100, 400], 'penalty': ['l2'], 'loss': ['hinge','squared_hinge']},]
gs=GridSearchCV(clf6,param_grid).fit(X_train_PCA,y_train)
gs.best_estimator_, gs.best_score_, gs.best_params_

(LinearSVC(C=0.1, loss='hinge', max_iter=5000),
 0.31966292134831464,
 {'C': 0.1, 'loss': 'hinge', 'penalty': 'l2'})

## Ensemble methods

In [ ]:
dataset = pd.read_csv("dataset_clone.csv")
y=dataset.iloc[:,-1]
X=dataset.iloc[:,:-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [201]:
ensemble1=RandomForestClassifier().fit(X_train, y_train)
ensemble1.score(X_test, y_test)

0.46616541353383456

In [202]:
ensemble2=AdaBoostClassifier().fit(X_train, y_train)
ensemble2.score(X_test, y_test)

0.39849624060150374

In [203]:
ensemble3=GradientBoostingClassifier().fit(X_train, y_train)
ensemble3.score(X_test,y_test)

0.40601503759398494

In [90]:
red4=PCA(0.95)
X_train_PCA=red4.fit_transform(X_train)
X_test_PCA=red4.transform(X_test)

In [91]:
ensemble1=RandomForestClassifier().fit(X_train_PCA, y_train)
ensemble1.score(X_test_PCA, y_test)

0.42105263157894735

In [92]:
ensemble2=AdaBoostClassifier().fit(X_train_PCA, y_train)
ensemble2.score(X_test_PCA, y_test)

0.3684210526315789

In [93]:
ensemble3=GradientBoostingClassifier().fit(X_train_PCA, y_train)
ensemble3.score(X_test_PCA,y_test)

0.39849624060150374

In [94]:
dataset = pd.read_csv("dataset_clone.csv")
y=dataset.iloc[:,-1]
X=dataset.iloc[:,:-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [98]:
ensemble4 = lgb.LGBMClassifier()
param_grid = {
    'num_leaves': [25,28,31,33,35],
    'learning_rate': [0.001, 0.01, 0.1, 1],
    'n_estimators': [10000] #max
}
gbm = GridSearchCV(ensemble4, param_grid, cv=3)
gbm.fit(X_train, y_train)
gbm.best_estimator_, gbm.best_score_, gbm.best_index_, gbm.best_params_

(LGBMClassifier(n_estimators=10000, num_leaves=25),
 0.39298178818452945,
 10,
 {'learning_rate': 0.1, 'n_estimators': 10000, 'num_leaves': 25})

In [101]:
ensemble5 = lgb.LGBMClassifier()
param_grid = {
    'num_leaves': [21,22,23,24,25,26],
    'learning_rate': [0.01, 0.1],
    'n_estimators': [100]
}
gbm2 = GridSearchCV(ensemble5, param_grid, cv=3)
gbm2.fit(X_train, y_train)
gbm2.best_estimator_, gbm2.best_score_, gbm2.best_index_, gbm2.best_params_

(LGBMClassifier(num_leaves=21),
 0.36039723332698775,
 6,
 {'learning_rate': 0.1, 'n_estimators': 100, 'num_leaves': 21})

In [ ]:
gbm = lgb.LGBMClassifier(num_leaves=25,
                        learning_rate=0.01,
                        n_estimators=2000,
                        max_depth=7)
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric=['multi_error','multiclass'],
        early_stopping_rounds=70)
gbm.score(X_test, y_test)

In [ ]:
red4=PCA(.95)
X_train_PCA=red4.fit_transform(X_train)
X_test_PCA=red4.transform(X_test)
ensemble6 = lgb.LGBMClassifier()
param_grid = {
    'num_leaves': [22,21,20,19,18],
    'learning_rate': [0.01,0.001],
    'n_estimators': [100],
    'max_depth': [7,8,9,10]
}
gbm3 = GridSearchCV(ensemble5, param_grid, cv=3)
gbm3.fit(X_train_P, y_train)
gbm3.best_estimator_, gbm3.best_score_, gbm3.best_index_, gbm3.best_params_

## Naive bayes

In [204]:
nb1=GaussianNB().fit(X_train, y_train)
nb1.score(X_test,y_test)

0.3007518796992481

In [205]:
nb2=ComplementNB().fit(X_train, y_train)
nb2.score(X_test,y_test)

0.3157894736842105

In [221]:
nb4=BernoulliNB(alpha=0.075, binarize=0, fit_prior=True).fit(X_train, y_train)
nb4.score(X_test,y_test)

0.5639097744360902

In [284]:
clf6 = NearestCentroid().fit(X_train, y_train)
clf6.score(X_test,y_test)

0.42857142857142855

In [ ]:
red4=PCA(.95)
X_train_PCA=red4.fit_transform(X_train)
X_test_PCA=red4.transform(X_test)

In [75]:
nb1=GaussianNB().fit(X_train, y_train)
nb1.score(X_test,y_test)

0.2932330827067669

In [76]:
nb2=ComplementNB().fit(X_train, y_train)
nb2.score(X_test,y_test)

0.2932330827067669

In [77]:
nb4=BernoulliNB(alpha=0.075, binarize=0, fit_prior=True).fit(X_train, y_train)
nb4.score(X_test,y_test)

0.47368421052631576

In [78]:
clf6 = NearestCentroid().fit(X_train, y_train)
clf6.score(X_test,y_test)

0.23308270676691728

In [ ]:
#Couldn't run this test in some computers...

#nca = NeighborhoodComponentsAnalysis().fit_transform(X_train, y_train) 
#nca.transform(X_test,y_test)
#knn = KNeighborsClassifier(n_neighbors=7).fit(X_train, y_train)
#knn.score(X_test, y_test)

## Preprocessing: FFT (fast fourier transform)

In [4]:
X_fourier=fft(X)
X_fourier_abs=np.abs(X_fourier)
X_f=pd.DataFrame.from_records(X_fourier_abs)
X_f.shape
#from here X_train X_test... Are fourier transformed

(441, 12000)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_f, y, test_size=0.30)

In [106]:
clf_fft= LogisticRegression(random_state=0, penalty='none').fit(X_train, y_train)
clf_fft.score(X_test, y_test)

0.6616541353383458

In [107]:
clf2_fft = SVC()
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]
gs_fft=GridSearchCV(clf2_fft,param_grid).fit(X,y)
gs_fft.cv_results_
gs_fft.best_score_, gs_fft.best_params_, gs_fft.best_estimator_, gs_fft.best_index_

(0.408197139938713,
 {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'},
 SVC(C=100, gamma=0.0001),
 9)

In [226]:
clf3_fft = LinearSVC()
param_grid = [{'C': [0.5, 1, 10, 100, 1000], 'loss': ['hinge', 'squared_hinge']},]
gs2_fft=GridSearchCV(clf3,param_grid).fit(X,y)
gs2_fft.best_score_, gs2_fft.best_params_, gs2_fft.best_estimator_, gs2_fft.best_index_

(0.34466292134831467,
 {'C': 10, 'loss': 'hinge'},
 LinearSVC(C=10, loss='hinge'),
 4)

In [227]:
#KNC n_neighbors iterators to test the best number of it
sc=0
iteration=0
for neig in range(1,200):
    clf3 = KNeighborsClassifier(n_neighbors=neig).fit(X_train, y_train)
    if sc<clf3.score(X_test,y_test):
        sc=clf3.score(X_test,y_test)
        iteration=neig
print (sc, iteration)

0.631578947368421 7


In [56]:
red5=PCA(.90)
X_train_PCA=red5.fit_transform(X_train)
X_test_PCA=red5.transform(X_test)
sc=0
iteration=0
for neig in range(1,200):
    clf3 = KNeighborsClassifier(n_neighbors=neig).fit(X_train_PCA, y_train)
    if sc<clf3.score(X_test_PCA,y_test):
        sc=clf3.score(X_test_PCA,y_test)
        iteration=neig
print (sc, iteration)

0.6917293233082706 20


In [232]:
clf_kn_fft= KNeighborsClassifier(7).fit(X_train, y_train)
clf_kn_fft.score(X_test, y_test)

0.631578947368421

In [251]:
nb4=BernoulliNB(alpha=0.075).fit(X_train, y_train)
nb4.score(X_test,y_test)

0.24060150375939848

In [21]:
clf4_fft = LogisticRegression(random_state=0)
params=([{'penalty': ['none'], 'solver': ['newton-cg', 'lbfgs', 'sag','saga']},
        {'penalty': ['elasticnet'], 'solver': ['saga']},
        {'penalty': ['l2'], 'solver': ['newton-cg', 'lbfgs', 'sag','saga','liblinear']},
        {'penalty': ['l1'], 'solver': ['liblinear', 'saga']},])

gs=GridSearchCV(clf4_fft,params).fit(X_train,y_train)
gs.best_score_, gs.best_estimator_, gs.best_params_

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not 

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


(0.6365415124272872,
 LogisticRegression(penalty='none', random_state=0, solver='sag'),
 {'penalty': 'none', 'solver': 'sag'})

In [27]:
clf5_fft=LogisticRegression(penalty='none', solver='sag', max_iter=10000).fit(X_train, y_train)
clf5_fft.score(X_test, y_test)

0.7593984962406015

In [ ]:
clf6_fft = LogisticRegression(random_state=0, max_iter=10000)
params=([{'penalty': ['none'], 'solver': ['newton-cg', 'lbfgs', 'sag','saga']},
        {'penalty': ['l2'], 'solver': ['newton-cg', 'lbfgs', 'sag','saga','liblinear'], 'C': [0.1, 1, 100]},
        {'penalty': ['l1'], 'solver': ['liblinear', 'saga'], 'C': [0.1, 1, 100]},])

gs=GridSearchCV(clf6_fft,params).fit(X_train,y_train)
gs.best_score_, gs.best_estimator_, gs.best_params_

In [32]:
clf7_fft=LogisticRegression(random_state=0, max_iter=10000, penalty='none', solver='sag', multi_class='multinomial').fit(X_train, y_train)
clf8_fft=LogisticRegression(random_state=0, max_iter=10000, penalty='none', solver='sag', multi_class='ovr').fit(X_train, y_train)
clf7_fft.score(X_test, y_test), clf8_fft.score(X_test, y_test)

0.7669172932330827

In [34]:
clf10_fft=LogisticRegression(random_state=0, max_iter=10000, penalty='elasticnet', solver='saga')
params=([{'multi_class': ['ovr', 'multinomial'], 'l1_ratio':[0, 0.25, 0.5, 0.75, 1]}])
gs=GridSearchCV(clf10_fft,params).fit(X_train,y_train)
gs.best_score_, gs.best_estimator_, gs.best_params_

(0.6204124801692227,
 LogisticRegression(l1_ratio=0, max_iter=10000, multi_class='ovr',
                    penalty='elasticnet', random_state=0, solver='saga'),
 {'l1_ratio': 0, 'multi_class': 'ovr'})

In [110]:
red4=PCA(.95)
X_train_PCA=red4.fit_transform(X_train)
X_test_PCA=red4.transform(X_test)
clf8_fft=LogisticRegression(random_state=0, max_iter=10000, penalty='none', solver='sag', multi_class='ovr').fit(X_train_PCA, y_train)
clf8_fft.score(X_test_PCA, y_test)

0.6766917293233082

In [ ]:
clf11_fft_PCA = LogisticRegression(random_state=0, max_iter=10000)
params=([{'penalty': ['none'], 'solver': ['newton-cg', 'lbfgs', 'sag','saga']},
        {'penalty': ['elasticnet'], 'solver': ['saga'], 'C': [0.1, 1, 100]},
        {'penalty': ['l2'], 'solver': ['newton-cg', 'sag','saga','liblinear'], 'C': [0.1, 1, 100]},
        {'penalty': ['l1'], 'solver': ['liblinear', 'saga'], 'C': [0.1, 1, 100]},])

gs=GridSearchCV(clf11_fft_PCA,params).fit(X_train_PCA,y_train)
gs.best_score_, gs.best_estimator_, gs.best_params_

In [60]:
lgbm_fft= lgb.LGBMClassifier()
param_grid = {
    'num_leaves': [25,28,31,33,35],
    'learning_rate': [0.001, 0.01, 0.1, 1],
    'n_estimators': [10000] #max
}
gbm = GridSearchCV(lgbm_fft, param_grid, cv=3)
gbm.fit(X_train, y_train)
gbm.best_estimator_, gbm.best_score_, gbm.best_index_, gbm.best_params_

(LGBMClassifier(n_estimators=10000, num_leaves=25),
 0.386382384669078,
 10,
 {'learning_rate': 0.1, 'n_estimators': 10000, 'num_leaves': 25})

In [7]:
red4=PCA(.95)
X_train_PCA=red4.fit_transform(X_train)
X_test_PCA=red4.transform(X_test)

lgbm_fft_PCA= lgb.LGBMClassifier()
param_grid = {
    'num_leaves': [25,28,31,33,35],
    'learning_rate': [0.001, 0.01, 0.1, 1],
    'n_estimators': [10000] #max
}
gbm = GridSearchCV(lgbm_fft_PCA, param_grid)
gbm.fit(X_train_PCA, y_train)
gbm.best_estimator_, gbm.best_score_, gbm.best_index_, gbm.best_params_

(LGBMClassifier(learning_rate=0.001, n_estimators=10000, num_leaves=25),
 0.6915730693571928,
 0,
 {'learning_rate': 0.001, 'n_estimators': 10000, 'num_leaves': 25})

In [61]:
red4=PCA(.95)
X_train_PCA=red4.fit_transform(X_train)
X_test_PCA=red4.transform(X_test)

gbm = lgb.LGBMClassifier(num_leaves=15,
                        learning_rate=0.001,
                        n_estimators=20000)#                        max_depth=10)             
gbm.fit(X_train_PCA, y_train,
eval_set=[(X_test_PCA, y_test)],
eval_metric=['multi_error','multiclass'],
early_stopping_rounds=100)


gbm.score(X_test_PCA, y_test)

[1]	valid_0's multi_error: 0.714286	valid_0's multi_logloss: 1.38901
Training until validation scores don't improve for 100 rounds
[2]	valid_0's multi_error: 0.714286	valid_0's multi_logloss: 1.38876
[3]	valid_0's multi_error: 0.714286	valid_0's multi_logloss: 1.38852
[4]	valid_0's multi_error: 0.714286	valid_0's multi_logloss: 1.38826
[5]	valid_0's multi_error: 0.714286	valid_0's multi_logloss: 1.38801
[6]	valid_0's multi_error: 0.714286	valid_0's multi_logloss: 1.38777
[7]	valid_0's multi_error: 0.714286	valid_0's multi_logloss: 1.38752
[8]	valid_0's multi_error: 0.714286	valid_0's multi_logloss: 1.38728
[9]	valid_0's multi_error: 0.714286	valid_0's multi_logloss: 1.38703
[10]	valid_0's multi_error: 0.714286	valid_0's multi_logloss: 1.38678
[11]	valid_0's multi_error: 0.714286	valid_0's multi_logloss: 1.38654
[12]	valid_0's multi_error: 0.714286	valid_0's multi_logloss: 1.3863
[13]	valid_0's multi_error: 0.714286	valid_0's multi_logloss: 1.38607
[14]	valid_0's multi_error: 0.714286	v

[144]	valid_0's multi_error: 0.691729	valid_0's multi_logloss: 1.3612
[145]	valid_0's multi_error: 0.691729	valid_0's multi_logloss: 1.36098
[146]	valid_0's multi_error: 0.691729	valid_0's multi_logloss: 1.36087
[147]	valid_0's multi_error: 0.691729	valid_0's multi_logloss: 1.36069
[148]	valid_0's multi_error: 0.691729	valid_0's multi_logloss: 1.3604
[149]	valid_0's multi_error: 0.691729	valid_0's multi_logloss: 1.36032
[150]	valid_0's multi_error: 0.691729	valid_0's multi_logloss: 1.36007
[151]	valid_0's multi_error: 0.691729	valid_0's multi_logloss: 1.35999
[152]	valid_0's multi_error: 0.684211	valid_0's multi_logloss: 1.35975
[153]	valid_0's multi_error: 0.676692	valid_0's multi_logloss: 1.35954
[154]	valid_0's multi_error: 0.676692	valid_0's multi_logloss: 1.35935
[155]	valid_0's multi_error: 0.676692	valid_0's multi_logloss: 1.35927
[156]	valid_0's multi_error: 0.676692	valid_0's multi_logloss: 1.35906
[157]	valid_0's multi_error: 0.676692	valid_0's multi_logloss: 1.35888
[158]	va

[282]	valid_0's multi_error: 0.654135	valid_0's multi_logloss: 1.33698
[283]	valid_0's multi_error: 0.646617	valid_0's multi_logloss: 1.33687
[284]	valid_0's multi_error: 0.654135	valid_0's multi_logloss: 1.33669
[285]	valid_0's multi_error: 0.654135	valid_0's multi_logloss: 1.3366
[286]	valid_0's multi_error: 0.646617	valid_0's multi_logloss: 1.33644
[287]	valid_0's multi_error: 0.654135	valid_0's multi_logloss: 1.33631
[288]	valid_0's multi_error: 0.661654	valid_0's multi_logloss: 1.33611
[289]	valid_0's multi_error: 0.661654	valid_0's multi_logloss: 1.33601
[290]	valid_0's multi_error: 0.661654	valid_0's multi_logloss: 1.33582
[291]	valid_0's multi_error: 0.654135	valid_0's multi_logloss: 1.33559
[292]	valid_0's multi_error: 0.661654	valid_0's multi_logloss: 1.33557
[293]	valid_0's multi_error: 0.654135	valid_0's multi_logloss: 1.33538
[294]	valid_0's multi_error: 0.654135	valid_0's multi_logloss: 1.33516
[295]	valid_0's multi_error: 0.654135	valid_0's multi_logloss: 1.33517
[296]	v

0.3609022556390977